### Imports

In [1]:
import os
import tensorflow as tf
from PIL import Image
from nets import nets_factory   
import numpy as np

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Char set 

In [2]:
number = ['0','1','2','3','4','5','6','7','8','9']
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
ALPHABET = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

comb1 = number + alphabet
comb2 = number + alphabet + ALPHABET

simple = number + ['a','b','c','g']
print(comb2)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


### Parameters

In [3]:
# number+ lower: lr = 0.003  train for 6000   size: 10000/500
# number + lower + upper: lr = 0.001  train for 6000+  size: 10000/500


CAHR_SET_LEN = len(comb2) #################################################################################################
IMAGE_HEIGHT, IMAGE_WIDTH = 60, 100
BATCH_SIZE = 25
TFRECORD_FILE = 'captcha/train.tfrecords'

x = tf.placeholder(tf.float32,[None,224,224])
y0 = tf.placeholder(tf.float32,[None])
y1 = tf.placeholder(tf.float32,[None])
y2 = tf.placeholder(tf.float32,[None])
y3 = tf.placeholder(tf.float32,[None])

lr = tf.Variable(0.001, dtype = tf.float32)

### Auxiliary function

In [4]:
def read_and_decode(filename):
    filename_queue = tf.train.string_input_producer([filename])
    reader = tf.TFRecordReader()
    
    # get filenames(labels)
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example,
                                      features = {
                                      'image': tf.FixedLenFeature([],tf.string),
                                      'label0': tf.FixedLenFeature([],tf.int64),
                                      'label1': tf.FixedLenFeature([],tf.int64),
                                      'label2': tf.FixedLenFeature([],tf.int64),
                                      'label3': tf.FixedLenFeature([],tf.int64) 
                                 })
    
    
    image = tf.decode_raw(features['image'],tf.uint8)   
      
    image = tf.reshape(image,[224,224])    
    image = tf.cast(image,tf.float32) / 255.0
    image = tf.subtract(image,0.5)
    image = tf.multiply(image,2.0)
    
    label0 = tf.cast(features['label0'],tf.int32)
    label1 = tf.cast(features['label1'],tf.int32)
    label2 = tf.cast(features['label2'],tf.int32)
    label3 = tf.cast(features['label3'],tf.int32)

    return image, label0, label1, label2, label3

### Train

In [5]:
image, label0, label1, label2, label3 = read_and_decode(TFRECORD_FILE)

# divide batches(shuffle)
image_batch, label_batch0, label_batch1, label_batch2, label_batch3 = tf.train.shuffle_batch(
        [image, label0, label1,label2,label3], batch_size = BATCH_SIZE,
        capacity = 50000, min_after_dequeue = 10000, num_threads = 1)

train_network_fn = nets_factory.get_network_fn(
    'alexnet_v2',
    num_classes = CAHR_SET_LEN,
    weight_decay = 0.0005,
    is_training = True)

with tf.Session() as sess:    

    #inputs: a tensor of size[batch_size, height, width,channels]
    
    X = tf.reshape(x,[BATCH_SIZE, 224, 224, 1])
    
    # forward pass 
    logits0, logits1, logits2, logits3, end_points = train_network_fn(X)

    # one-hot encoder
    one_hot_labels0 = tf.one_hot(indices = tf.cast(y0,tf.int32), depth = CAHR_SET_LEN)
    one_hot_labels1 = tf.one_hot(indices = tf.cast(y1,tf.int32), depth = CAHR_SET_LEN)
    one_hot_labels2 = tf.one_hot(indices = tf.cast(y2,tf.int32), depth = CAHR_SET_LEN)
    one_hot_labels3 = tf.one_hot(indices = tf.cast(y3,tf.int32), depth = CAHR_SET_LEN)

    # compute loss
    loss0 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits0, labels = one_hot_labels0))
    loss1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits1, labels = one_hot_labels1))
    loss2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits2, labels = one_hot_labels2))
    loss3 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits3, labels = one_hot_labels3))    
    total_loss = (loss0 + loss1 + loss2 + loss3) / 4.0

    # define optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate = lr).minimize(total_loss)

    # accuracy
    correct_prediction0 = tf.equal(tf.argmax(one_hot_labels0, 1), tf.argmax(logits0, 1))
    accuracy0 = tf.reduce_mean(tf.cast(correct_prediction0, tf.float32))

    correct_prediction1 = tf.equal(tf.argmax(one_hot_labels1, 1), tf.argmax(logits1, 1))
    accuracy1 = tf.reduce_mean(tf.cast(correct_prediction1, tf.float32))

    correct_prediction2 = tf.equal(tf.argmax(one_hot_labels2, 1), tf.argmax(logits2, 1))
    accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32))

    correct_prediction3 = tf.equal(tf.argmax(one_hot_labels3, 1), tf.argmax(logits3, 1))
    accuracy3 = tf.reduce_mean(tf.cast(correct_prediction3, tf.float32))

    losses = []
    accs0, accs1,accs2,accs3 = [],[],[],[]
    # init training 
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())    
    coord = tf.train.Coordinator()     # create coordinator for threads management   
    threads = tf.train.start_queue_runners(sess = sess, coord = coord)

    for i in range(20001):        
        b_image, b_label0, b_label1, b_label2, b_label3 = sess.run([image_batch, label_batch0, label_batch1, label_batch2, label_batch3])        
        
        sess.run(optimizer, feed_dict = {x:b_image, y0: b_label0, y1:b_label1, y2:b_label2, y3:b_label3})
        
        if i % 20 == 0:
            
            if i % 5000 == 0:
                sess.run(tf.assign(lr, lr/3))
                
            acc0, acc1, acc2, acc3, loss_ = sess.run([accuracy0,accuracy1,accuracy2,accuracy3, total_loss], feed_dict = {x:b_image, y0: b_label0, y1:b_label1, y2:b_label2, y3:b_label3})
            learning_rate = sess.run(lr)
            print("Iter: %d Loss:%.3f  Accuracy:%.2f, %.2f, %.2f, %.2f Learning Rate: %.4f" % (i, loss_, acc0, acc1,acc2,acc3, learning_rate))
            
            if loss_ < 0.13:
                break
            
            # sample
#             losses.append(loss_)
#             accs0.append(acc0)
#             accs1.append(acc1)
#             accs2.append(acc2)
#             accs3.append(acc3)
            
#             if i == 20000:
#                 saver.save(sess,'./captcha/models/crack_captcha.model', global_step = i)
#                 break 
    saver.save(sess,'./captcha/models/crack_captcha.model', global_step = i)            
    coord.request_stop()    
    coord.join(threads)

Iter: 0 Loss:7.782  Accuracy:0.12, 0.16, 0.12, 0.12 Learning Rate: 0.0003
Iter: 20 Loss:4.133  Accuracy:0.00, 0.00, 0.04, 0.04 Learning Rate: 0.0003
Iter: 40 Loss:4.122  Accuracy:0.04, 0.04, 0.00, 0.00 Learning Rate: 0.0003
Iter: 60 Loss:4.131  Accuracy:0.04, 0.04, 0.04, 0.04 Learning Rate: 0.0003
Iter: 80 Loss:4.132  Accuracy:0.00, 0.00, 0.00, 0.00 Learning Rate: 0.0003
Iter: 100 Loss:4.130  Accuracy:0.08, 0.00, 0.00, 0.04 Learning Rate: 0.0003
Iter: 120 Loss:4.134  Accuracy:0.00, 0.04, 0.00, 0.00 Learning Rate: 0.0003
Iter: 140 Loss:4.138  Accuracy:0.04, 0.04, 0.00, 0.00 Learning Rate: 0.0003
Iter: 160 Loss:4.122  Accuracy:0.00, 0.00, 0.00, 0.00 Learning Rate: 0.0003
Iter: 180 Loss:4.126  Accuracy:0.08, 0.04, 0.04, 0.00 Learning Rate: 0.0003
Iter: 200 Loss:4.128  Accuracy:0.00, 0.04, 0.04, 0.00 Learning Rate: 0.0003
Iter: 220 Loss:4.120  Accuracy:0.08, 0.04, 0.04, 0.04 Learning Rate: 0.0003
Iter: 240 Loss:4.124  Accuracy:0.04, 0.04, 0.04, 0.04 Learning Rate: 0.0003
Iter: 260 Loss:4.1

Iter: 2160 Loss:3.945  Accuracy:0.12, 0.00, 0.00, 0.04 Learning Rate: 0.0003
Iter: 2180 Loss:3.926  Accuracy:0.08, 0.00, 0.00, 0.04 Learning Rate: 0.0003
Iter: 2200 Loss:3.976  Accuracy:0.08, 0.08, 0.04, 0.00 Learning Rate: 0.0003
Iter: 2220 Loss:4.001  Accuracy:0.08, 0.00, 0.00, 0.04 Learning Rate: 0.0003
Iter: 2240 Loss:3.978  Accuracy:0.04, 0.00, 0.00, 0.00 Learning Rate: 0.0003
Iter: 2260 Loss:3.968  Accuracy:0.08, 0.00, 0.04, 0.04 Learning Rate: 0.0003
Iter: 2280 Loss:3.948  Accuracy:0.08, 0.00, 0.04, 0.00 Learning Rate: 0.0003
Iter: 2300 Loss:3.898  Accuracy:0.08, 0.08, 0.00, 0.00 Learning Rate: 0.0003
Iter: 2320 Loss:3.841  Accuracy:0.16, 0.04, 0.04, 0.00 Learning Rate: 0.0003
Iter: 2340 Loss:3.892  Accuracy:0.16, 0.00, 0.00, 0.04 Learning Rate: 0.0003
Iter: 2360 Loss:3.852  Accuracy:0.24, 0.00, 0.04, 0.00 Learning Rate: 0.0003
Iter: 2380 Loss:3.904  Accuracy:0.04, 0.00, 0.08, 0.04 Learning Rate: 0.0003
Iter: 2400 Loss:3.825  Accuracy:0.08, 0.00, 0.00, 0.08 Learning Rate: 0.0003

Iter: 4300 Loss:1.956  Accuracy:0.80, 0.24, 0.28, 0.48 Learning Rate: 0.0003
Iter: 4320 Loss:2.188  Accuracy:0.88, 0.24, 0.28, 0.24 Learning Rate: 0.0003
Iter: 4340 Loss:2.110  Accuracy:0.80, 0.24, 0.20, 0.28 Learning Rate: 0.0003
Iter: 4360 Loss:1.601  Accuracy:0.92, 0.36, 0.40, 0.44 Learning Rate: 0.0003
Iter: 4380 Loss:2.087  Accuracy:0.76, 0.24, 0.20, 0.36 Learning Rate: 0.0003
Iter: 4400 Loss:1.607  Accuracy:0.76, 0.52, 0.36, 0.44 Learning Rate: 0.0003
Iter: 4420 Loss:1.789  Accuracy:0.88, 0.28, 0.24, 0.52 Learning Rate: 0.0003
Iter: 4440 Loss:1.863  Accuracy:0.80, 0.40, 0.20, 0.44 Learning Rate: 0.0003
Iter: 4460 Loss:1.680  Accuracy:0.68, 0.36, 0.32, 0.52 Learning Rate: 0.0003
Iter: 4480 Loss:1.866  Accuracy:0.64, 0.36, 0.44, 0.32 Learning Rate: 0.0003
Iter: 4500 Loss:1.868  Accuracy:0.76, 0.48, 0.28, 0.52 Learning Rate: 0.0003
Iter: 4520 Loss:1.734  Accuracy:0.76, 0.56, 0.44, 0.32 Learning Rate: 0.0003
Iter: 4540 Loss:1.676  Accuracy:0.80, 0.32, 0.24, 0.32 Learning Rate: 0.0003

Iter: 6440 Loss:0.955  Accuracy:0.92, 0.52, 0.68, 0.72 Learning Rate: 0.0001
Iter: 6460 Loss:0.636  Accuracy:0.88, 0.72, 0.64, 0.72 Learning Rate: 0.0001
Iter: 6480 Loss:0.918  Accuracy:0.84, 0.64, 0.60, 0.60 Learning Rate: 0.0001
Iter: 6500 Loss:0.767  Accuracy:0.88, 0.44, 0.64, 0.72 Learning Rate: 0.0001
Iter: 6520 Loss:0.756  Accuracy:0.92, 0.76, 0.68, 0.60 Learning Rate: 0.0001
Iter: 6540 Loss:0.612  Accuracy:0.76, 0.88, 0.84, 0.68 Learning Rate: 0.0001
Iter: 6560 Loss:0.528  Accuracy:0.96, 0.76, 0.80, 0.84 Learning Rate: 0.0001
Iter: 6580 Loss:0.664  Accuracy:0.80, 0.68, 0.52, 0.88 Learning Rate: 0.0001
Iter: 6600 Loss:0.694  Accuracy:0.76, 0.72, 0.68, 0.88 Learning Rate: 0.0001
Iter: 6620 Loss:0.744  Accuracy:0.92, 0.64, 0.68, 0.72 Learning Rate: 0.0001
Iter: 6640 Loss:0.741  Accuracy:0.92, 0.76, 0.76, 0.68 Learning Rate: 0.0001
Iter: 6660 Loss:0.646  Accuracy:0.84, 0.80, 0.76, 0.88 Learning Rate: 0.0001
Iter: 6680 Loss:0.643  Accuracy:0.84, 0.84, 0.60, 0.76 Learning Rate: 0.0001

Iter: 8580 Loss:0.523  Accuracy:0.84, 0.72, 0.92, 0.68 Learning Rate: 0.0001
Iter: 8600 Loss:0.508  Accuracy:0.96, 0.64, 0.76, 0.92 Learning Rate: 0.0001
Iter: 8620 Loss:0.440  Accuracy:0.76, 0.84, 0.92, 0.84 Learning Rate: 0.0001
Iter: 8640 Loss:0.504  Accuracy:0.92, 0.80, 0.84, 0.84 Learning Rate: 0.0001
Iter: 8660 Loss:0.387  Accuracy:0.92, 0.88, 0.76, 0.88 Learning Rate: 0.0001
Iter: 8680 Loss:0.451  Accuracy:0.92, 0.76, 0.92, 0.76 Learning Rate: 0.0001
Iter: 8700 Loss:0.367  Accuracy:0.96, 0.80, 0.88, 0.92 Learning Rate: 0.0001
Iter: 8720 Loss:0.557  Accuracy:0.92, 0.72, 0.84, 0.84 Learning Rate: 0.0001
Iter: 8740 Loss:0.501  Accuracy:0.96, 0.68, 0.72, 0.88 Learning Rate: 0.0001
Iter: 8760 Loss:0.357  Accuracy:0.96, 0.88, 0.76, 0.88 Learning Rate: 0.0001
Iter: 8780 Loss:0.455  Accuracy:0.92, 0.92, 0.84, 0.76 Learning Rate: 0.0001
Iter: 8800 Loss:0.353  Accuracy:1.00, 0.80, 0.72, 0.88 Learning Rate: 0.0001
Iter: 8820 Loss:0.322  Accuracy:1.00, 0.72, 0.92, 0.88 Learning Rate: 0.0001

Iter: 10700 Loss:0.354  Accuracy:0.96, 0.92, 0.92, 0.84 Learning Rate: 0.0000
Iter: 10720 Loss:0.305  Accuracy:0.92, 0.92, 0.96, 0.84 Learning Rate: 0.0000
Iter: 10740 Loss:0.436  Accuracy:0.92, 0.84, 0.84, 0.88 Learning Rate: 0.0000
Iter: 10760 Loss:0.267  Accuracy:0.92, 0.84, 0.88, 0.92 Learning Rate: 0.0000
Iter: 10780 Loss:0.465  Accuracy:0.88, 0.92, 0.64, 0.92 Learning Rate: 0.0000
Iter: 10800 Loss:0.194  Accuracy:0.96, 0.88, 0.96, 0.92 Learning Rate: 0.0000
Iter: 10820 Loss:0.233  Accuracy:1.00, 0.80, 0.92, 1.00 Learning Rate: 0.0000
Iter: 10840 Loss:0.219  Accuracy:0.92, 0.96, 0.92, 1.00 Learning Rate: 0.0000
Iter: 10860 Loss:0.220  Accuracy:1.00, 0.96, 0.88, 0.96 Learning Rate: 0.0000
Iter: 10880 Loss:0.331  Accuracy:0.96, 0.80, 0.88, 0.96 Learning Rate: 0.0000
Iter: 10900 Loss:0.403  Accuracy:0.88, 0.96, 0.88, 0.88 Learning Rate: 0.0000
Iter: 10920 Loss:0.290  Accuracy:0.96, 0.92, 0.76, 0.84 Learning Rate: 0.0000
Iter: 10940 Loss:0.282  Accuracy:0.96, 0.92, 0.84, 0.72 Learning

Iter: 12820 Loss:0.148  Accuracy:0.96, 1.00, 0.92, 0.96 Learning Rate: 0.0000
Iter: 12840 Loss:0.318  Accuracy:1.00, 0.80, 0.92, 0.84 Learning Rate: 0.0000
Iter: 12860 Loss:0.354  Accuracy:0.96, 0.88, 0.80, 0.92 Learning Rate: 0.0000
Iter: 12880 Loss:0.208  Accuracy:0.92, 0.92, 0.88, 0.96 Learning Rate: 0.0000
Iter: 12900 Loss:0.222  Accuracy:0.96, 0.92, 0.92, 0.92 Learning Rate: 0.0000
Iter: 12920 Loss:0.241  Accuracy:0.88, 0.88, 0.96, 0.92 Learning Rate: 0.0000
Iter: 12940 Loss:0.234  Accuracy:0.96, 0.88, 0.84, 1.00 Learning Rate: 0.0000
Iter: 12960 Loss:0.343  Accuracy:0.96, 0.80, 0.88, 0.88 Learning Rate: 0.0000
Iter: 12980 Loss:0.214  Accuracy:1.00, 0.96, 1.00, 0.92 Learning Rate: 0.0000
Iter: 13000 Loss:0.214  Accuracy:1.00, 0.80, 0.88, 0.96 Learning Rate: 0.0000
Iter: 13020 Loss:0.273  Accuracy:0.88, 0.88, 0.88, 0.96 Learning Rate: 0.0000
Iter: 13040 Loss:0.404  Accuracy:0.92, 0.76, 1.00, 0.88 Learning Rate: 0.0000
Iter: 13060 Loss:0.228  Accuracy:0.96, 0.92, 0.88, 0.92 Learning

Iter: 14940 Loss:0.253  Accuracy:0.96, 0.76, 0.84, 0.92 Learning Rate: 0.0000
Iter: 14960 Loss:0.176  Accuracy:1.00, 1.00, 0.80, 0.84 Learning Rate: 0.0000
Iter: 14980 Loss:0.093  Accuracy:1.00, 0.96, 1.00, 0.96 Learning Rate: 0.0000
Iter: 15000 Loss:0.154  Accuracy:1.00, 0.92, 0.92, 0.96 Learning Rate: 0.0000
Iter: 15020 Loss:0.202  Accuracy:0.96, 0.92, 0.76, 0.96 Learning Rate: 0.0000
Iter: 15040 Loss:0.288  Accuracy:0.96, 0.84, 0.76, 0.96 Learning Rate: 0.0000
Iter: 15060 Loss:0.277  Accuracy:0.92, 0.92, 0.84, 0.88 Learning Rate: 0.0000
Iter: 15080 Loss:0.129  Accuracy:0.96, 0.96, 1.00, 0.92 Learning Rate: 0.0000
Iter: 15100 Loss:0.231  Accuracy:0.96, 0.96, 0.88, 0.92 Learning Rate: 0.0000
Iter: 15120 Loss:0.220  Accuracy:0.96, 0.84, 0.88, 0.88 Learning Rate: 0.0000
Iter: 15140 Loss:0.199  Accuracy:0.96, 1.00, 0.88, 0.96 Learning Rate: 0.0000
Iter: 15160 Loss:0.166  Accuracy:0.92, 0.88, 0.96, 1.00 Learning Rate: 0.0000
Iter: 15180 Loss:0.209  Accuracy:0.96, 0.92, 0.80, 1.00 Learning

KeyboardInterrupt: 